# 如何在链式结构中使用工具
在本指南中，我们将介绍创建调用[工具](/docs/concepts/tools/)的链（Chains）和代理（Agents）的基本方法。工具几乎可以是任何形式——API、函数、数据库等。工具使我们能够将模型的能力扩展到仅输出文本/消息之外。使用模型与工具配合的关键在于正确提示模型并解析其响应，使其选择正确的工具并为这些工具提供恰当的输入。

## 安装设置
本指南需要安装以下软件包：

In [ ]:
%pip install --upgrade --quiet langchain

若想在 [LangSmith](https://docs.smith.langchain.com/) 中追踪您的运行记录，请取消注释并设置以下环境变量：

In [ ]:
import getpass
import os

# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## 创建一个工具
首先，我们需要创建一个可供调用的工具。在本示例中，我们将通过函数创建一个自定义工具。有关创建自定义工具的更多信息，请参阅[此指南](/docs/how_to/custom_tools)。

In [6]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [2]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(first_int: int, second_int: int) -> int - Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [3]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

## 链条
如果我们知道只需要固定次数地使用某个工具，就可以为此创建一个链。让我们创建一个简单的链，仅用于对用户指定的数字进行乘法运算。
![链条](../../static/img/tool_chain.svg)
### 工具/函数调用与大型语言模型（LLM）结合使用工具最可靠的方法之一是通过[工具调用](/docs/concepts/tool_calling/)API（有时也称为函数调用）。此功能仅适用于明确支持工具调用的模型。您可以在[此处](/docs/integrations/chat/)查看支持工具调用的模型列表，并通过[本指南](/docs/how_to/function_calling)了解更多关于如何使用工具调用的信息。
首先我们将定义我们的模型和工具。我们将从单一工具`multiply`开始。
import ChatModelTabs from "@theme/ChatModelTabs";
<ChatModelTabs customVarName="llm"/>


In [7]:
# | echo: false
# | output: false

from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

我们将使用 `bind_tools` 将工具的定义作为每次模型调用的参数传入，以便模型在适当时机调用该工具：

In [8]:
llm_with_tools = llm.bind_tools([multiply])

当模型调用工具时，这将在输出的 `AIMessage.tool_calls` 属性中显示：

In [9]:
msg = llm_with_tools.invoke("whats 5 times forty two")
msg.tool_calls

[{'name': 'multiply',
  'args': {'first_int': 5, 'second_int': 42},
  'id': 'call_cCP9oA3tRz7HDrjFn1FdmDaG'}]

查看 [LangSmith 追踪记录](https://smith.langchain.com/public/81ff0cbd-e05b-4720-bf61-2c9807edb708/r)。

### 调用工具
太好了！我们已经能够生成工具调用了。但如果想要实际调用工具该怎么办呢？为此，我们需要将生成的工具参数传递给我们的工具。举个简单的例子，我们只需提取第一个工具调用的参数：

In [12]:
from operator import itemgetter

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
chain.invoke("What's four times 23")

92

查看 [LangSmith 追踪记录](https://smith.langchain.com/public/16bbabb9-fc9b-41e5-a33d-487c42df4f85/r)。

## 代理
当我们明确知道针对任何用户输入所需使用的工具具体序列时，链式调用非常有效。但在某些应用场景中，工具的使用次数取决于输入内容。这种情况下，我们需要让模型自行决定工具的使用次数和调用顺序。[智能体](/docs/tutorials/agents)正是为此而设计。
LangChain 内置了多种针对不同使用场景优化的智能代理。查阅所有[代理类型](/docs/concepts/agents)的相关说明。
我们将使用[工具调用代理](https://python.langchain.com/api_reference/langchain/agents/langchain.agents.tool_calling_agent.base.create_tool_calling_agent.html)，这通常是最可靠的类型，也是大多数用例的推荐选择。
![代理](../../static/img/tool_agent.svg)

In [13]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [14]:
# Get the prompt to use - can be replaced with any prompt that includes variables "agent_scratchpad" and "input"!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


代理的另一大优势在于它们能轻松整合多种工具的使用。

In [15]:
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [multiply, add, exponentiate]

In [16]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [17]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

借助代理，我们可以提出需要任意多次使用工具的问题：

In [18]:
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15
Invoking: `multiply` with `{'first_int': 243, 'second_int': 15}`


3645
Invoking: `exponentiate` with `{'base': 405, 'exponent': 2}`


13286025The result of taking 3 to the fifth power is 243. 

The sum of twelve and three is 15. 

Multiplying 243 by 15 gives 3645. 

Finally, squaring 3645 gives 13286025.

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': 'The result of taking 3 to the fifth power is 243. \n\nThe sum of twelve and three is 15. \n\nMultiplying 243 by 15 gives 3645. \n\nFinally, squaring 3645 gives 13286025.'}

查看 [LangSmith 追踪记录](https://smith.langchain.com/public/eeeb27a4-a2f8-4f06-a3af-9c983f76146c/r)。